In [1]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=int(os.environ["ACT_AS"]),
)

In [11]:
import ipywidgets as widgets
import pandas as pd

def market_selector():
    state = client.state()
    open_markets = [market.definition.name for market in state.markets.values() if market.definition.open]
    return widgets.Combobox(
        placeholder='Choose a market',
        options=open_markets,
        description='Market:',
        ensure_option=True,
        disabled=False
    )

def get_market_orders(market_name):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)
    if market_id is None:
        return pd.DataFrame()
    orders = state.markets[market_id].orders
    return pd.DataFrame(orders)

def clear_market_orders(market_name):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)
    if market_id is None:
        return
    client.out(market_id)
    return f"Cleared orders for {market_name}"


orders_widget = widgets.interact(get_market_orders, market_name=market_selector())

out_widget = widgets.interact_manual(clear_market_orders, market_name=market_selector())


interactive(children=(Combobox(value='', description='Market:', ensure_option=True, options=('5', 'test_1', 'h…

interactive(children=(Combobox(value='', continuous_update=False, description='Market:', ensure_option=True, o…

In [2]:
import pandas as pd
import qgrid
# Your existing code to get trades
state = client.state()
market_name = "test_2"
market_id = state.market_name_to_id[market_name]
if not state.markets[market_id].hasFullTradeHistory:
    client.get_full_trade_history(market_id)
    state = client.state()
trades = state.markets[market_id].trades

qgrid.show_grid(pd.DataFrame(trades), show_toolbar=True)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…